In [7]:
!pip install datasets transformers ipywidgets huggingface_hub 

In [8]:
pip install --upgrade notebook

Note: you may need to restart the kernel to use updated packages.


In [9]:
from huggingface_hub import hf_hub_download

# Try downloading a model or dataset to test the connection
file_path = hf_hub_download("bert-base-uncased", "config.json")

print(f"File downloaded to: {file_path}")

File downloaded to: /Users/saeah/.cache/huggingface/hub/models--bert-base-uncased/snapshots/86b5e0934494bd15c9632b12f734a8a67f723594/config.json


In [10]:
from huggingface_hub import notebook_login

notebook_login()

In [11]:
!brew install git-lfs

==> Auto-updating Homebrew...
Adjust how often this is run with HOMEBREW_AUTO_UPDATE_SECS or disable with
HOMEBREW_NO_AUTO_UPDATE. Hide these hints with HOMEBREW_NO_ENV_HINTS (see `man brew`).
==> Homebrew collects anonymous analytics.
Read the analytics documentation (and how to opt-out) here:
  https://docs.brew.sh/Analytics
No analytics have been recorded yet (nor will be during this `brew` run).

==> Homebrew is run entirely by unpaid volunteers. Please consider donating:
  https://github.com/Homebrew/brew#donations

==> Auto-updated Homebrew!
Updated 2 taps (homebrew/core and homebrew/cask).
==> New Formulae
adapterremoval  ggh             lazyjj          reuse           yamlfix
bacon-ls        git-mob         libpostal       rink            yices2
bagels          go@1.23         mac             rustywind       yor
cfnctl          hcledit         mummer          sby             zimfw
comrak          identme         ramalama        sql-formatter
gauth           jupytext        rat

In [13]:
from transformers.utils import send_example_telemetry

send_example_telemetry("question_answering_notebook", framework="pytorch")

In [14]:
# This flag is the difference between SQUAD v1 or 2 (if you're using another dataset, it indicates if impossible
# answers are allowed or not).
squad_v2 = False
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

In [15]:
!pip install evaluate

In [16]:
# from datasets import load_dataset, load_metric
from datasets import load_dataset
from evaluate import load # load_metric is now load

In [17]:
datasets = load_dataset("squad_v2" if squad_v2 else "squad")

In [18]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [19]:
datasets["train"][0]

{'id': '5733be284776f41900661182',
 'title': 'University_of_Notre_Dame',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}

In [20]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [21]:
show_random_elements(datasets["train"])

,id,title,context,question,answers
0,5727a584ff5b5019007d91e4,Crucifixion_of_Jesus,"In Johannine ""agent Christology"" the submission of Jesus to crucifixion is a sacrifice made as an agent of God or servant of God, for the sake of eventual victory. This builds on the salvific theme of the Gospel of John which begins in John 1:29 with John the Baptist's proclamation: ""The Lamb of God who takes away the sins of the world"". Further reinforcement of the concept is provided in Revelation 21:14 where the ""lamb slain but standing"" is the only one worthy of handling the scroll (i.e. the book) containing the names of those who are to be saved.",What theme is central is the Gospel of John in resurrection?,"{'text': ['salvific theme'], 'answer_start': [183]}"
1,56e7b1e900c9c71400d77506,Daylight_saving_time,"Western France, Spain, and other areas skew time zones and shift clocks, in effect observing DST in winter with an extra hour in summer. Nome, Alaska, is at 165°24′ W longitude, which is just west of center of the idealized Samoa Time Zone (165° W), but Nome observes Alaska Time (135° W) with DST, so it is slightly more than two hours ahead of the sun in winter and three in summer. Double daylight saving time has been used on occasion; for example, it was used in some European countries during and shortly after World War II when it was referred to as ""Double Summer Time"". See British Double Summer Time and Central European Midsummer Time for details.",What term was used in Britain for double daylight saving time?,"{'text': ['Double Summer Time'], 'answer_start': [558]}"
2,572a1ca66aef05140015529e,East_Prussia,"The Teutonic Order lost eastern Prussia when Grand Master Albert of Brandenburg-Ansbach converted to Lutheranism and secularized the Prussian branch of the Teutonic Order in 1525. Albert established himself as the first duke of the Duchy of Prussia and a vassal of the Polish crown by the Prussian Homage. Walter von Cronberg, the next Grand Master, was enfeoffed with the title to Prussia after the Diet of Augsburg in 1530, but the Order never regained possession of the territory. In 1569 the Hohenzollern prince-electors of the Margraviate of Brandenburg became co-regents with Albert's son, the feeble-minded Albert Frederick.",Who converted to Lutheranism and secularized the Prussian branch of the Teutonic Order?,"{'text': ['Albert of Brandenburg-Ansbach'], 'answer_start': [58]}"
3,572e8b9edfa6aa1500f8d104,Richard_Feynman,"Feynman was elected a Foreign Member of the Royal Society (ForMemRS) in 1965. At this time in the early 1960s, Feynman exhausted himself by working on multiple major projects at the same time, including a request, while at Caltech, to ""spruce up"" the teaching of undergraduates. After three years devoted to the task, he produced a series of lectures that eventually became The Feynman Lectures on Physics. He wanted a picture of a drumhead sprinkled with powder to show the modes of vibration at the beginning of the book. Concerned over the connections to drugs and rock and roll that could be made from the image, the publishers changed the cover to plain red, though they included a picture of him playing drums in the foreword. The Feynman Lectures on Physics occupied two physicists, Robert B. Leighton and Matthew Sands, as part-time co-authors for several years. Even though the books were not adopted by most universities as textbooks, they continue to sell well because they provide a deep understanding of physics. Many of his lectures and miscellaneous talks were turned into other books, including The Character of Physical Law, QED: The Strange Theory of Light and Matter, Statistical Mechanics, Lectures on Gravitation, and the Feynman Lectures on Computation.","In his book, Feynman was shown playing what instrument?","{'text': ['drums'], 'answer_start': [710]}"
4,56e08070231d4119001ac203,Saint_Helena,"Saint Helena has the international calling code +290 which, since 2006, Tristan da Cunha shares. Saint Hele

Preprocessing the train data

In [22]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [23]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [24]:
tokenizer("What is your name?", "My name is Sylvain.")

{'input_ids': [101, 2054, 2003, 2115, 2171, 1029, 102, 2026, 2171, 2003, 25353, 22144, 2378, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [25]:
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

In [26]:
for i, example in enumerate(datasets["train"]):
    if len(tokenizer(example["question"], example["context"])["input_ids"]) > 384:
        break
example = datasets["train"][i]

In [27]:
len(tokenizer(example["question"], example["context"])["input_ids"])

396

In [28]:
len(tokenizer(example["question"], example["context"], max_length=max_length, truncation="only_second")["input_ids"])

384

In [29]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    stride=doc_stride
)

In [30]:
[len(x) for x in tokenized_example["input_ids"]]

[384, 157]

In [31]:
for x in tokenized_example["input_ids"][:2]:
    print(tokenizer.decode(x))

[CLS] how many wins does the notre dame men ' s basketball team have? [SEP] the men ' s basketball team has over 1, 600 wins, one of only 12 schools who have reached that mark, and have appeared in 28 ncaa tournaments. former player austin carr holds the record for most points scored in a single game of the tournament with 61. although the team has never won the ncaa tournament, they were named by the helms athletic foundation as national champions twice. the team has orchestrated a number of upsets of number one ranked teams, the most notable of which was ending ucla ' s record 88 - game winning streak in 1974. the team has beaten an additional eight number - one teams, and those nine wins rank second, to ucla ' s 10, all - time in wins against the top team. the team plays in newly renovated purcell pavilion ( within the edmund p. joyce center ), which reopened for the beginning of the 2009 – 2010 season. the team is coached by mike brey, who, as of the 2014 – 15 season, his fifteenth

In [32]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    stride=doc_stride
)
print(tokenized_example["offset_mapping"][0][:100])

[(0, 0), (0, 3), (4, 8), (9, 13), (14, 18), (19, 22), (23, 28), (29, 33), (34, 37), (37, 38), (38, 39), (40, 50), (51, 55), (56, 60), (60, 61), (0, 0), (0, 3), (4, 7), (7, 8), (8, 9), (10, 20), (21, 25), (26, 29), (30, 34), (35, 36), (36, 37), (37, 40), (41, 45), (45, 46), (47, 50), (51, 53), (54, 58), (59, 61), (62, 69), (70, 73), (74, 78), (79, 86), (87, 91), (92, 96), (96, 97), (98, 101), (102, 106), (107, 115), (116, 118), (119, 121), (122, 126), (127, 138), (138, 139), (140, 146), (147, 153), (154, 160), (161, 165), (166, 171), (172, 175), (176, 182), (183, 186), (187, 191), (192, 198), (199, 205), (206, 208), (209, 210), (211, 217), (218, 222), (223, 225), (226, 229), (230, 240), (241, 245), (246, 248), (248, 249), (250, 258), (259, 262), (263, 267), (268, 271), (272, 277), (278, 281), (282, 285), (286, 290), (291, 301), (301, 302), (303, 307), (308, 312), (313, 318), (319, 321), (322, 325), (326, 330), (330, 331), (332, 340), (341, 351), (352, 354), (355, 363), (364, 373), (374,

In [33]:
first_token_id = tokenized_example["input_ids"][0][1]
offsets = tokenized_example["offset_mapping"][0][1]
print(tokenizer.convert_ids_to_tokens([first_token_id])[0], example["question"][offsets[0]:offsets[1]])

how How


In [34]:
sequence_ids = tokenized_example.sequence_ids()
print(sequence_ids)

[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [35]:
answers = example["answers"]
start_char = answers["answer_start"][0]
end_char = start_char + len(answers["text"][0])

# Start token index of the current span in the text.
token_start_index = 0
while sequence_ids[token_start_index] != 1:
    token_start_index += 1

# End token index of the current span in the text.
token_end_index = len(tokenized_example["input_ids"][0]) - 1
while sequence_ids[token_end_index] != 1:
    token_end_index -= 1

# Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
offsets = tokenized_example["offset_mapping"][0]
if (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
    # Move the token_start_index and token_end_index to the two ends of the answer.
    # Note: we could go after the last offset if the answer is the last word (edge case).
    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
        token_start_index += 1
    start_position = token_start_index - 1
    while offsets[token_end_index][1] >= end_char:
        token_end_index -= 1
    end_position = token_end_index + 1
    print(start_position, end_position)
else:
    print("The answer is not in this feature.")

23 26


In [36]:
print(tokenizer.decode(tokenized_example["input_ids"][0][start_position: end_position+1]))
print(answers["text"][0])

over 1, 600
over 1,600


In [37]:
pad_on_right = tokenizer.padding_side == "right"

In [38]:
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [39]:
features = prepare_train_features(datasets['train'][:5])

In [40]:
tokenized_datasets = datasets.map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)

Fine-tuning the model

In [41]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [42]:
!pip show accelerate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Name: accelerate
Version: 1.3.0
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: zach.mueller@huggingface.co
License: Apache
Location: /Users/saeah/dl_env/lib/python3.9/site-packages
Requires: huggingface-hub, numpy, packaging, psutil, pyyaml, safetensors, torch
Required-by: 


In [43]:
!pip install accelerate --upgrade

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [44]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-squad",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)

/Users/saeah/dl_env/lib/python3.9/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [45]:
from transformers import default_data_collator

data_collator = default_data_collator

In [46]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

/var/folders/53/8n97sd_j4s3529sgst4wc6jc0000gn/T/ipykernel_11247/1580614439.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [48]:
trainer.save_model("test-squad-trained")

events.out.tfevents.1739582444.Saeahs-MacBook-Air.local.11247.0:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

In [49]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("test-squad-trained")
model = AutoModelForQuestionAnswering.from_pretrained("test-squad-trained")

In [50]:
# Create a Question-Answering pipeline
qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer)

Device set to use mps:0


In [51]:
# Ask Questions
context = "Hugging Face is a company that creates natural language processing tools."
question = "What does Hugging Face create?"

result = qa_pipeline(question=question, context=context)
print(result)


{'score': 0.9613043069839478, 'start': 39, 'end': 72, 'answer': 'natural language processing tools'}


In [52]:
# Ask Questions
context = "Python is a popular programming language used in web development, data science, and artificial intelligence. Many beginners start with online courses, tutorials, and books."
question = "What is the best way to learn Python?"

result = qa_pipeline(question=question, context=context)
print(result)

{'score': 0.32835084199905396, 'start': 135, 'end': 171, 'answer': 'online courses, tutorials, and books'}


In [53]:
# Ask Questions
context = "Albert Einstein developed the theory of relativity, which consists of special relativity and general relativity. Special relativity describes how time and space are relative to the observer’s motion, while general relativity explains gravity as the warping of spacetime by mass."
question = "Can you summarize Einstein’s theory of relativity?"

result = qa_pipeline(question=question, context=context)
print(result)

{'score': 0.024027813225984573, 'start': 30, 'end': 50, 'answer': 'theory of relativity'}


In [54]:
# Ask Questions
context = "Artificial intelligence is transforming industries, automating repetitive tasks, and improving efficiency. Some experts predict that AI will replace certain jobs, while others believe it will create new opportunities."
question = "What will be the impact of AI on jobs in the next 10 years?"

result = qa_pipeline(question=question, context=context)
print(result)


{'score': 0.3690001368522644, 'start': 192, 'end': 216, 'answer': 'create new opportunities'}


In [55]:
# Ask Questions
context = "Writing is an essential skill used in communication, business, and creative fields. Many authors suggest reading regularly, practicing writing daily, and seeking feedback to improve."
question = "How can I improve my writing skills?"

result = qa_pipeline(question=question, context=context)
print(result)

{'score': 0.42676687240600586, 'start': 154, 'end': 170, 'answer': 'seeking feedback'}


In [56]:
# Ask Questions
context = "Many scientists have made groundbreaking contributions. Isaac Newton formulated the laws of motion and gravity. Albert Einstein developed relativity. Charles Darwin proposed evolution by natural selection."
question = "Why is climate change happening?"

result = qa_pipeline(question=question, context=context)
print(result)

{'score': 0.1031956598162651, 'start': 187, 'end': 204, 'answer': 'natural selection'}


In [57]:
# Ask Questions
context = "A black hole forms when a massive star collapses under its own gravity, creating a singularity where the gravitational pull is so strong that not even light can escape."
question = "How do black holes form?"

result = qa_pipeline(question=question, context=context)
print(result)

{'score': 0.8094052672386169, 'start': 19, 'end': 70, 'answer': 'when a massive star collapses under its own gravity'}


In [58]:
# Ask Questions
context = "Artificial intelligence raises ethical concerns regarding bias in algorithms, job displacement, and data privacy. Many experts are calling for responsible AI development."
question = "What are the ethical concerns with AI?"

result = qa_pipeline(question=question, context=context)
print(result)


{'score': 0.8956823945045471, 'start': 58, 'end': 112, 'answer': 'bias in algorithms, job displacement, and data privacy'}


In [59]:
# Ask Questions
context = "Viruses and bacteria are microorganisms, but they differ significantly. Bacteria are single-celled organisms that can live independently, while viruses require a host cell to replicate."
question = "What is the difference between a virus and bacteria?"

result = qa_pipeline(question=question, context=context)
print(result)


{'score': 0.3967583477497101, 'start': 45, 'end': 70, 'answer': 'they differ significantly'}


In [60]:
# Ask Questions
context = "Job interviews allow employers to assess candidates. Many hiring managers recommend preparing in advance, dressing professionally, and demonstrating confidence."
question = "How do I make a good impression in a job interview?"

result = qa_pipeline(question=question, context=context)
print(result)

{'score': 0.17804531753063202, 'start': 15, 'end': 51, 'answer': 'allow employers to assess candidates'}


Questions expected that ChatGPT can answer likely to fail but BERT can handle well

In [61]:
# Ask Questions
context = "The Nobel Prize in Physics for the year 1921 was awarded to Albert Einstein for his services to theoretical physics, particularly for his discovery of the law of the photoelectric effect."
question = "Who won the Nobel Prize in Physics in 1921?"

result = qa_pipeline(question=question, context=context)
print(result)

{'score': 0.9967067241668701, 'start': 60, 'end': 75, 'answer': 'Albert Einstein'}


In [62]:
# Ask Questions
context = "Paris is the capital and most populous city of France, located in the Île-de-France region."
question = "What is the capital of France?"

result = qa_pipeline(question=question, context=context)
print(result)

{'score': 0.9883519411087036, 'start': 0, 'end': 5, 'answer': 'Paris'}


In [63]:
# Ask Questions
context = "World War II ended in 1945 after Germany surrendered in May and Japan surrendered in September."
question = "What year did World War II end?"

result = qa_pipeline(question=question, context=context)
print(result)

{'score': 0.9898930191993713, 'start': 22, 'end': 26, 'answer': '1945'}


In [64]:
# Ask Questions
context = "Oxygen is the element with the atomic number 8 in the periodic table."
question = "Which element has the atomic number 8?"

result = qa_pipeline(question=question, context=context)
print(result)

{'score': 0.997136652469635, 'start': 0, 'end': 6, 'answer': 'Oxygen'}


In [65]:
# Ask Questions
context = "The Mona Lisa was painted by Leonardo da Vinci during the Renaissance period and is housed in the Louvre Museum in Paris."
question = "Who painted the Mona Lisa?"

result = qa_pipeline(question=question, context=context)
print(result)

{'score': 0.9965728521347046, 'start': 29, 'end': 46, 'answer': 'Leonardo da Vinci'}


In [66]:
# Ask Questions
context = "Mount Everest, standing at 8,848 meters above sea level, is the tallest mountain in the world."
question = "What is the tallest mountain in the world?"

result = qa_pipeline(question=question, context=context)
print(result)

{'score': 0.978338360786438, 'start': 0, 'end': 13, 'answer': 'Mount Everest'}


In [67]:
# Ask Questions
context = "At sea level, water boils at 100 degrees Celsius (212 degrees Fahrenheit) under normal atmospheric pressure."
question = "What is the boiling point of water at sea level?"

result = qa_pipeline(question=question, context=context)
print(result)

{'score': 0.4989550709724426, 'start': 29, 'end': 48, 'answer': '100 degrees Celsius'}


In [68]:
# Ask Questions
context = "The dystopian novel '1984' was written by George Orwell and published in 1949."
question = "Who wrote the novel '1984'?"

result = qa_pipeline(question=question, context=context)
print(result)

{'score': 0.9981210231781006, 'start': 42, 'end': 55, 'answer': 'George Orwell'}


In [69]:
# Ask Questions
context = "The speed of light in a vacuum is approximately 299,792 kilometers per second (186,282 miles per second)."
question = "What is the speed of light?"

result = qa_pipeline(question=question, context=context)
print(result)

{'score': 0.7844801545143127, 'start': 48, 'end': 77, 'answer': '299,792 kilometers per second'}


In [70]:
# Ask Questions
context = "Water is a molecule composed of two hydrogen atoms and one oxygen atom, giving it the chemical formula H₂O."
question = "What is the chemical formula for water?"

result = qa_pipeline(question=question, context=context)
print(result)

{'score': 0.987666666507721, 'start': 103, 'end': 106, 'answer': 'H₂O'}


In [71]:
# Ask Questions
context = "Mathematics is the study of numbers, structures, and change. Arithmetic is a fundamental branch of mathematics, dealing with operations such as addition, subtraction, multiplication, and division."
question = "What is 2 + 2?"

result = qa_pipeline(question=question, context=context)
print(result)

{'score': 0.778788149356842, 'start': 61, 'end': 71, 'answer': 'Arithmetic'}


In [37]:
# # Share the model on Hugging Face Hub
# # Push the model
# model.push_to_hub("sawah521/test-squad-trained")
# tokenizer.push_to_hub("sawah521/test-squad-trained")


No files have been modified since last commit. Skipping to prevent empty commit.
No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/sawah521/test-squad-trained/commit/c824be42c9378fd8a68cf2507f578f019a19dbcf', commit_message='Upload tokenizer', commit_description='', oid='c824be42c9378fd8a68cf2507f578f019a19dbcf', pr_url=None, repo_url=RepoUrl('https://huggingface.co/sawah521/test-squad-trained', endpoint='https://huggingface.co', repo_type='model', repo_id='sawah521/test-squad-trained'), pr_revision=None, pr_num=None)

In [72]:
# After pushing models to Hugging Face, we can load the model anywhere like this:
model = AutoModelForQuestionAnswering.from_pretrained("test-squad-trained")
tokenizer = AutoTokenizer.from_pretrained("test-squad-trained")

In [73]:
!pip install transformers datasets evaluate

In [53]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

# Load the model and tokenizer from the Hugging Face Model Hub
model_name = "test-squad-trained"  #distilbert-base-uncased-finetuned-squad"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

In [74]:
from datasets import load_dataset

# Load the SQuAD dataset
squad = load_dataset("squad")
validation_data = squad["validation"]

In [75]:
from transformers import pipeline

# Create a question-answering pipeline
qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer)

# Generate predictions
predictions = []
references = []

for example in validation_data:
    context = example["context"]
    question = example["question"]
    ground_truth = example["answers"]["text"]
    
    # Generate the model's prediction
    prediction = qa_pipeline({"context": context, "question": question})["answer"]
    
    # Append predictions and references for evaluation
    predictions.append({"id": example["id"], "prediction_text": prediction})
    references.append({"id": example["id"], "answers": example["answers"]})

Device set to use mps:0
/Users/saeah/dl_env/lib/python3.9/site-packages/transformers/pipelines/question_answering.py:391: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(


In [76]:
from evaluate import load

# Load the SQuAD evaluation metric
squad_metric = load("squad")

# Compute F1 and EM
results = squad_metric.compute(predictions=predictions, references=references)
print("F1 Score:", results["f1"])
print("Exact Match (EM):", results["exact_match"])

F1 Score: 85.6769095082034
Exact Match (EM): 77.70104068117313


Evaluation

In [77]:
import torch

for batch in trainer.get_eval_dataloader():
    break
batch = {k: v.to(trainer.args.device) for k, v in batch.items()}
with torch.no_grad():
    output = trainer.model(**batch)
output.keys()

odict_keys(['loss', 'start_logits', 'end_logits'])

In [78]:
output.start_logits.shape, output.end_logits.shape

(torch.Size([16, 384]), torch.Size([16, 384]))

In [79]:
output.start_logits.argmax(dim=-1), output.end_logits.argmax(dim=-1)

(tensor([ 46,  57,  78,  54, 118, 162,  72,  35, 162,  34,  73,  41,  80,  91,
         156,  35], device='mps:0'),
 tensor([ 47,  58,  81,  44, 118, 110,  75,  37, 110,  36,  76,  42,  83,  94,
         158,  35], device='mps:0'))

In [80]:
n_best_size = 20

In [81]:
import numpy as np

start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": "" # We need to find a way to get back the original substring corresponding to the answer in the context
                }
            )

In [82]:
# Re-process the validation set 
def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [83]:
validation_features = datasets["validation"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=datasets["validation"].column_names
)

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [84]:
raw_predictions = trainer.predict(validation_features)

In [85]:
validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))

In [86]:
max_answer_length = 30

In [87]:
start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
offset_mapping = validation_features[0]["offset_mapping"]
# The first feature comes from the first example. For the more general case, we will need to be match the example_id to
# an example index
context = datasets["validation"][0]["context"]

# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
        # to part of the input_ids that are not in the context.
        if (
            start_index >= len(offset_mapping)
            or end_index >= len(offset_mapping)
            or offset_mapping[start_index] is None
            or offset_mapping[end_index] is None
        ):
            continue
        # Don't consider answers with a length that is either < 0 or > max_answer_length.
        if end_index < start_index or end_index - start_index + 1 > max_answer_length:
            continue
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            start_char = offset_mapping[start_index][0]
            end_char = offset_mapping[end_index][1]
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": context[start_char: end_char]
                }
            )

valid_answers = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[:n_best_size]
valid_answers

[{'score': 18.451855, 'text': 'Denver Broncos'},
 {'score': 15.582927,
  'text': 'Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers'},
 {'score': 13.305171, 'text': 'Broncos'},
 {'score': 12.748223, 'text': 'Carolina Panthers'},
 {'score': 12.326754, 'text': 'Denver'},
 {'score': 12.212813,
  'text': 'American Football Conference (AFC) champion Denver Broncos'},
 {'score': 10.436242,
  'text': 'Broncos defeated the National Football Conference (NFC) champion Carolina Panthers'},
 {'score': 10.252956,
  'text': 'The American Football Conference (AFC) champion Denver Broncos'},
 {'score': 10.225332,
  'text': 'Denver Broncos defeated the National Football Conference (NFC)'},
 {'score': 9.860188,
  'text': 'Denver Broncos defeated the National Football Conference (NFC) champion Carolina'},
 {'score': 9.571446,
  'text': 'Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl tit

In [88]:
datasets["validation"][0]["answers"]

{'text': ['Denver Broncos', 'Denver Broncos', 'Denver Broncos'],
 'answer_start': [177, 177, 177]}

In [89]:
import collections

examples = datasets["validation"]
features = validation_features

example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

In [90]:
from tqdm.auto import tqdm

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        if not squad_v2:
            predictions[example["id"]] = best_answer["text"]
        else:
            answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
            predictions[example["id"]] = answer

    return predictions

In [91]:
final_predictions = postprocess_qa_predictions(datasets["validation"], validation_features, raw_predictions.predictions)

Post-processing 10570 example predictions split into 10784 features.


  0%|          | 0/10570 [00:00<?, ?it/s]

In [92]:
metric = load("squad_v2" if squad_v2 else "squad")

In [93]:
if squad_v2:
    formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in final_predictions.items()]
else:
    formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]
references = [{"id": ex["id"], "answers": ex["answers"]} for ex in datasets["validation"]]
metric.compute(predictions=formatted_predictions, references=references)

{'exact_match': 77.55912961210974, 'f1': 85.7218110399946}

In [94]:
trainer.push_to_hub()

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/sawah521/distilbert-base-uncased-finetuned-squad/commit/ddb0f370a74d8914c91809c843879570f87e7473', commit_message='End of training', commit_description='', oid='ddb0f370a74d8914c91809c843879570f87e7473', pr_url=None, repo_url=RepoUrl('https://huggingface.co/sawah521/distilbert-base-uncased-finetuned-squad', endpoint='https://huggingface.co', repo_type='model', repo_id='sawah521/distilbert-base-uncased-finetuned-squad'), pr_revision=None, pr_num=None)